# Start with example class from before

Also, define a function so we can print our vehicle out easily

In [1]:
class Vehicle:
    def __init__(self, mpg=None, tank_size=None, passengers=None, transmission=None):
        self.mpg = mpg
        self.tank_size = tank_size
        self.passengers = passengers
        self.transmission = transmission
        self.max_range = self.calculate_max_range()
        
    def calculate_max_range(self):
        if (self.mpg is None) or (self.tank_size is None):
            return None
        return self.mpg * self.tank_size
    
def print_vehicle(vehicle):
    print(f"Fuel efficiency: {vehicle.mpg} mpg")
    print(f"Fuel capacity: {vehicle.tank_size} gallons")
    print(f"Passenger capacity: {vehicle.passengers} passengers")
    print(f"Transmission: {vehicle.transmission}")
    print(f"Max range: {vehicle.max_range} miles")

In [2]:
sedan = Vehicle(mpg=35, tank_size=15, passengers=5, transmission="automatic")

print_vehicle(sedan)

Fuel efficiency: 35 mpg
Fuel capacity: 15 gallons
Passenger capacity: 5 passengers
Transmission: automatic
Max range: 525 miles


# Now let's say we want to do an electric car
Electric cars don't have gas tanks or miles per gallon ratings. Instead they have a battery which has some capacity and efficiency that determine how far they can travel. We could rewrite the who class, but it would be nice to reuse what we've already done. We can do this through `subclassing`.

In [3]:
class ElectricVehicle(Vehicle):
    def __init__(self, battery_capacity, battery_efficiency):
        self.battery_capacity = battery_capacity # in kWh
        self.battery_efficiency = battery_efficiency # in kWh/mile

To create a subclass, we define a new class `ElectricVehicle` and pass the parent class `Vehicle` as an argument. However, we aren't quite done yet.

In [4]:
volta = ElectricVehicle(battery_capacity=85, battery_efficiency=0.341)

print_vehicle(volta)

AttributeError: 'ElectricVehicle' object has no attribute 'mpg'

Currently, our `ElectricVehicle` is not `inheriting` the attributes of the `Vehicle` class. To do this, we need to initiate the parent class using a `super()` function.

In [122]:
class ElectricVehicle(Vehicle):
    def __init__(self, battery_capacity, battery_efficiency):
        super().__init__()
        self.battery_capacity = battery_capacity # in kWh
        self.battery_efficiency = battery_efficiency # in kWh/mile

In [123]:
volta = ElectricVehicle(battery_capacity=85, battery_efficiency=0.341)

print_vehicle(volta)

Fuel efficiency: None mpg
Fuel capacity: None gallons
Passenger capacity: None passengers
Transmission: None
Max range: None


Now our new instance has access to all of the variables and functions defined in the parent class. All of the variables have a `None` value because those were the defaults given when we defined `Vehicle`. We can change those values by adding arguments to our `ElectricVehicle` class and passing those to the `super()` function.

In [124]:
class ElectricVehicle(Vehicle):
    def __init__(self, battery_capacity, battery_efficiency, passengers, transmission):
        super().__init__(passengers=passengers, transmission=transmission)
        self.battery_capacity = battery_capacity # in kWh
        self.battery_efficiency = battery_efficiency # in kWh/mile
        

In [125]:
volta = ElectricVehicle(battery_capacity=85, battery_efficiency=0.341, passengers=5, transmission="automatic")

print_vehicle(volta)

Fuel efficiency: None mpg
Fuel capacity: None gallons
Passenger capacity: 5 passengers
Transmission: automatic
Max range: None


We have populated the `passengers` and `transmission` fields, but there is a problem. Our `max_range` variable is defined by calling the `calculate_max_range()` function, which the `mpg` and `tank_size` as arguments. To prevent errors, we provide some initial logic that if either of these values is `None`, the result is `None`, which is what we get here. To change this, we can override the `calculate_max_range()` function by redefining it using the battery parameters.

In [127]:
class ElectricVehicle(Vehicle):
    def __init__(self, battery_capacity, battery_efficiency, passengers=None, transmission=None):
        super().__init__(passengers=passengers, transmission=transmission)
        self.battery_capacity = battery_capacity # in kWh
        self.battery_efficiency = battery_efficiency # in kWh/mile
        
    def calculate_max_range(self):
        if (self.battery_capacity is None) or (self.battery_efficiency is None):
            return None
        return int(self.battery_capacity / self.battery_efficiency)
        

However, if we try to run this we will encounter an error.

In [128]:
volta = ElectricVehicle(battery_capacity=85, battery_efficiency=0.341, passengers=5, transmission="automatic")

print_vehicle(volta)

AttributeError: 'ElectricVehicle' object has no attribute 'battery_capacity'

The error arises from our order of operations. Here we have redefined the `calculate_max_range()` function, which overrides the existing function definition in `Vehicle`. When we called the `super()` function, the parent class has tried to define `max_range` by calling the updated `calculate_max_range()` function. Unfortunately, we haven't defined the `battery_capacity` attribute, which results in this error. We can easily correct it by moving the `super()` call to after we define our battery attributes.

In [129]:
class ElectricVehicle(Vehicle):
    def __init__(self, battery_capacity, battery_efficiency, passengers=None, transmission=None):
        self.battery_capacity = battery_capacity # in kWh
        self.battery_efficiency = battery_efficiency # in kWh/mile
        super().__init__(passengers=passengers, transmission=transmission)
        
    def calculate_max_range(self):
        if (self.battery_capacity is None) or (self.battery_efficiency is None):
            return None
        return int(self.battery_capacity / self.battery_efficiency)
        

In [130]:
volta = ElectricVehicle(battery_capacity=85, battery_efficiency=0.341, passengers=5, transmission="automatic")

print_vehicle(volta)

Fuel efficiency: None mpg
Fuel capacity: None gallons
Passenger capacity: 5 passengers
Transmission: automatic
Max range: 249


# Propagation of class changes
One benefit of subclassing is that you can make changes to the parent class and those changes will be inherited by the child class. Let's say we want our vehicle class to calculate the carbon footprint caused by consumption of a tank of fuel. The US Environmental Protection Agency estimates that 8.9 kg of CO2 are emitted for every gallon of gasoline burned (https://www.epa.gov/greenvehicles/tailpipe-greenhouse-gas-emissions-typical-passenger-vehicle).

In [132]:
class Vehicle:
    def __init__(self, mpg=None, tank_size=None, passengers=None, transmission=None):
        self.mpg = mpg
        self.tank_size = tank_size
        self.passengers = passengers
        self.transmission = transmission
        self.max_range = self.calculate_max_range()
        self.co2_per_tank = self.calculate_emissions()
        
    def calculate_max_range(self):
        if (self.mpg is None) or (self.tank_size is None):
            return None
        return self.mpg * self.tank_size
    
    def calculate_emissions(self):
        if (self.mpg is None) or (self.tank_size is None):
            return None
        return 8.9 * self.tank_size
    
def print_vehicle(vehicle):
    print(f"Fuel efficiency: {vehicle.mpg} mpg")
    print(f"Fuel capacity: {vehicle.tank_size} gallons")
    print(f"Passenger capacity: {vehicle.passengers} passengers")
    print(f"Transmission: {vehicle.transmission}")
    print(f"Max range: {vehicle.max_range} miles")
    print(f"CO2 emissions per tank: {vehicle.co2_per_tank} kg")

In [133]:
sedan = Vehicle(mpg=35, tank_size=15, passengers=5, transmission="automatic")

print_vehicle(sedan)

Fuel efficiency: 35 mpg
Fuel capacity: 15 gallons
Passenger capacity: 5 passengers
Transmission: automatic
Max range: 525
CO2 emissions per tank: 133.5 kg


Without changing anything in our `ElectricVehicle` class, we can run the definition code again and then all new instances will be updated with the new class attributes.

In [135]:
class ElectricVehicle(Vehicle):
    def __init__(self, battery_capacity, battery_efficiency, passengers=None, transmission=None):
        self.battery_capacity = battery_capacity # in kWh
        self.battery_efficiency = battery_efficiency # in kWh/mile
        super().__init__(passengers=passengers, transmission=transmission)
        
    def calculate_max_range(self):
        if (self.battery_capacity is None) or (self.battery_efficiency is None):
            return None
        return int(self.battery_capacity / self.battery_efficiency)


In [136]:
volta = ElectricVehicle(battery_capacity=85, battery_efficiency=0.341, passengers=5, transmission="automatic")

print_vehicle(volta)

Fuel efficiency: None mpg
Fuel capacity: None gallons
Passenger capacity: 5 passengers
Transmission: automatic
Max range: 249
CO2 emissions per tank: None kg
